## Neural Network NER

In [94]:
import numpy as np
from os import listdir
import pickle
from xml.dom.minidom import parse
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
from keras.models import Model, Input
from keras.layers import *
#from keras_contrib.layers import CRF

stopwords = set(stopwords.words("english"))
from evaluator import *

%matplotlib inline

## Functions Learner

In [76]:
def find_multi_token_entity(ents, token):
    keys = ents.keys()
    for k in keys:
        if token in k:
            return ents[k]

def load_data(datadir):
    '''
    Task: Load XML files in given directory, tokenize each sentence, and extract
    ground truth BIO labels for each token.
    
    Input: 
        datadir: A directory containing XML files
        
    Output: A directory containing the dataset. Dictionary key is sentence_id, and the 
            value is a list of token tuples (word, start, end, ground truth).
            
    Example: 
            >>> load_data('data/Train')
            {'DDI-DrugBank.d370.s0': [('as', 0, 1,'O'), ('differin', 3, 10,'B-brand'),
                     ('gel', 12, 14,'O'), ... , ('with', 343, 346, 'O'),
                     ('caution', 348, 354, 'O'), ('.', 355, 355, 'O')],
            'DDI-DrugBank.d370.s1': [('particular', 0, 9, 'O'), ('caution', 11, 17, 'O'),
                     ('should', 19, 24, 'O'), ... , ('differin', 130, 137, 'B-brand'),
                     ('gel', 139, 141, 'O'), ('.', 142, 142, 'O')], ... }
    '''
    
    dict_dataset = {}
    
    for f in listdir(datadir):
        tree = parse(datadir + "/" + f)
        
        sentences = tree.getElementsByTagName("sentence")
        for s in sentences: 
            sid = s.attributes["id"].value
            
            stext = s.attributes["text"].value
            #stext = stext.replace("-"," ") if used we lose beta-endorphin, if not use we lose calcium-rich tag
            
            ents = {}
            entities = s.getElementsByTagName("entity")
            for e in entities:
                offset = e.attributes["charOffset"].value
                offsets = offset.split(';')
                name = e.attributes["text"].value
                e_type = e.attributes["type"].value
                ents[name] = {"type": e_type, "offsets": []}
                for offset in offsets:
                    start = offset.split('-')[0]
                    end = offset.split('-')[1]
                    ents[name]["offsets"].append((start, end))
            
            punct = [",",";",":","?","!", "(", ")"] # removed "."
            tokens = word_tokenize(stext)
            tokens_cleaned = []
            for t in tokens:
                if t not in punct and t not in stopwords:
                    tokens_cleaned.append(t)
            
            tags = []
            tokens = []
            for t in tokens_cleaned:
                offsetFrom = stext.find(t)
                offsetTo = offsetFrom + len(t) - 1
                # 1-token entities
                if t in ents:
                    if (int(ents[t]["offsets"][0][0]) == offsetFrom):
                        tag = "B-"+ents[t]["type"] # TODO: ents after .?
                    else:
                        tag = "I-"+ents[t]["type"]                    
                else:
                    multi_token_ent = find_multi_token_entity(ents, t)
                    if multi_token_ent:
                        if (int(multi_token_ent["offsets"][0][0]) == offsetFrom):
                            tag = "B-"+multi_token_ent["type"] # TODO: ents after .?
                        else:
                            tag = "I-"+multi_token_ent["type"]
                    else:
                        tag = "O"
                tags.append(tag)
                tupl = (t, offsetFrom, offsetTo, tag)
                tokens.append(tupl)
            
            dict_dataset[sid] = tokens
        
    return dict_dataset

# -- TODO: handle multi-token entities # I think I've done it :D
# tricyclic antidepressants - 'O' should be Group
# chondroitin ABC lyase - 'O' should be drug_n
# heparinase III - 'O' should be drug_n
# hyaluronan lyase - 'O' should be drug_n
# Mercaptopurine/Azathioprine - 'O' should be drug
# muscle relaxants 

In [78]:
path_train = "../../labAHLT/data/train"
path_dev = "../../labAHLT/data/devel"

train_dataset = load_data(path_train)
devel_dataset = load_data(path_dev)

print(train_dataset['DDI-DrugBank.d661.s5'])

[('Interactions', 0, 11, 'O'), ('observed', 23, 30, 'O'), ('nondepolarizing', 43, 57, 'B-group'), ('muscle', 59, 64, 'I-group'), ('relaxants', 66, 74, 'I-group'), ('administered', 86, 97, 'O'), ('succession', 102, 111, 'O'), ('.', 112, 112, 'O')]


In [79]:
def create_index(dataset, max_length):
    '''
    Task: Create index dictionaries both for input (words) and output (labels) from given dataset
    Input: 
        dataset: dataset produced by load_data.
        max_length: maximum length of a sentence (longer sentences will be cut, shorter ones will be padded).
        
    Output: A dictionary where each key is an index name (e.g. "words", "labels"), and the value is a 
            dictionary mapping each word/label to a number. An entry with the value for maxlen is also stored
    Example: 
        >>> create_indx(traindata)
        {'words': {'<PAD>':0, '<UNK>':1, '11-day':2, 'murine':3, 'criteria':4,
                   'stroke':5, ... ,'levodopa':8511, 'terfenadine': 8512}
         'labels': {'<PAD>':0, 'B-group':1, 'B-drug_n':2, 'I-drug_n':3, 'O':4, 
                    'I-group':5, 'B-drug':6, 'I-drug':7, 'B-brand':8, 'I-brand':9}
         'maxlen': 100 }
    '''
    
    index_words = {'<PAD>':0, '<UNK>':1}
    i = 2
    
    index_labels = {'<PAD>':0}
    j = 1
    
    for key, item in dataset.items():
        for t in item:
            word = t[0].lower() # use lower case words? 
            tag = t[3]
            if word not in index_words:
                index_words[word] = i
                i += 1
            if tag not in index_labels:
                index_labels[tag] = j
                j += 1

    indexs = {'words': index_words, 'labels': index_labels, 'maxlen':max_length}
    
    return indexs    

In [80]:
idx = create_index(train_dataset, 100)
idx['labels']

{'<PAD>': 0,
 'O': 1,
 'B-brand': 2,
 'B-drug': 3,
 'B-group': 4,
 'I-group': 5,
 'I-drug': 6,
 'I-brand': 7,
 'B-drug_n': 8,
 'I-drug_n': 9}

In [81]:
def encode_words(dataset, idx):
    '''
    Task: Encode the words in a sentence dataset formed by lists of tokens into lists of indexes
          suitable for NN input.
    Input: 
        dataset: A dataset produced by load_data.
        idx: A dictionary produced by create_indexs, containing word and label indexes, as well
             as the maximum sentence length.
             
    Output: The dataset encoded as a list of sentence, each of them is a list of word indices.
            If the word is not in the index, <UNK> code is used. If the sentence is shorter than
            max_len it is padded with <PAD> code.
    Example: 
        >>> encode_words(traindata, idx)
            [ [6882 1049 4911 ... 0 0 0 ]
            [  2290 7548 8069 ... 0 0 0 ]
               ...
            [  2002 6582 7518 ... 0 0 0 ] ]
    '''
    max_length = idx['maxlen']
    seq = []
    for key, item in dataset.items():
        aux = []
        for t in item:
            w = str(t[0]).lower() # When using lower case words
            if w in idx['words']:
                i = idx['words'][w]
            else:
                i = idx['words']['<UNK>']
            aux.append(i)
        seq.append(aux)  
    
    seq_padded = pad_sequences(maxlen = max_length, sequences = seq, padding = 'post')
    
    return seq_padded

def encode_labels(dataset, idx):
    '''
    Task: Encode the ground truth labels in a dataset formed by lists of tokens into lists of indexes
        suitable for NN output.
    Input:
        dataset: A dataset produced by load_data.
        idx: A dictionary produced by create_index, containing word and label indexes, as well as the maximum length.
        
    Output: The dataset encoded as a list of sentence, each of them is a list of BIO label indices. If the sentence
            is shorter than max_len it is padded with <PAD> code. 
    
    Example :
     >>> encode_labels ( traindata , idx )
        [[ [4] [6] [4] [4] [4] [4] ... [0] [0] ]
        [  [4] [4] [8] [4] [6] [4] ... [0] [0] ]
          ...
        [
    '''
    max_length = idx['maxlen']
    seq = []
    for key, item in dataset.items():
        aux = []
        for t in item:
            w = t[3]
            i = idx['labels'][w]
            aux.append(i)
        seq.append(aux)
    seq_padded = pad_sequences(maxlen = max_length, sequences = seq, padding = 'post')
    seq_categ = [to_categorical(i, num_classes = 10) for i in seq_padded]  # 9 classes + 1 PAD
    
    return seq_padded, seq_categ

In [82]:
X_train = encode_words(train_dataset, idx)
Y, Y_train = encode_labels(train_dataset, idx)

X_dev = encode_words(devel_dataset, idx)
Ydev, Y_dev = encode_labels(devel_dataset, idx)

In [83]:
print(X_train)

[[   2    3    4 ...    0    0    0]
 [   2   12    3 ...    0    0    0]
 [  19   20   21 ...    0    0    0]
 ...
 [   2  101 3387 ...    0    0    0]
 [1159  910 5135 ...    0    0    0]
 [  19  910 5135 ...    0    0    0]]


In [84]:
print(Y[0])
print(Y_train[0][0:10])

[1 1 2 1 1 3 1 1 3 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [88]:
def build_network(idx):
    '''
    Task: Create network for the learner. 
    Input:
        idx: index dictionary with word/labels codes, plus maximum sentence length.
    Output: Returns a compiled Keras neural network with the specified layers
    '''
    
    #sizes
    n_words = len(idx['words'])
    n_labels = len(idx['labels'])#+1
    max_len = idx['maxlen']
    
    # create network layers
    inp = Input(shape=(max_len,))
    model = Embedding(input_dim=n_words + 1, output_dim = n_labels, input_length = max_len)(inp)
    model = Dropout(0.2)(model)
    model = Bidirectional(LSTM(units=100, return_sequences=True, recurrent_dropout=0.1))(model)
    out = TimeDistributed(Dense(n_labels, activation="softmax"))(model)
    
    # create and compile model
    model = Model(inp, out)
    
#     optimiz = Adam(lr=0.01, decay=1e-6)
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    
    return model

In [89]:
model = build_network(idx)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 100)]             0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 100, 10)           83050     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 10)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 100, 200)          88800     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 100, 10)           2010      
Total params: 173,860
Trainable params: 173,860
Non-trainable params: 0
_________________________________________________________________


In [90]:
history = model.fit(X_train, np.array(Y_train),
                   batch_size=32,
                   epochs=2,
                   verbose=1, 
                   validation_data=(X_dev, np.array(Y_dev)))

Epoch 1/2
170/170 [==============================] - 38s 97ms/step - loss: 0.7678 - accuracy: 0.8721 - val_loss: 0.1363 - val_accuracy: 0.9643
Epoch 2/2
170/170 [==============================] - 15s 86ms/step - loss: 0.1299 - accuracy: 0.9675 - val_loss: 0.1156 - val_accuracy: 0.9714


In [91]:
def save_model_and_indexes(model, idx, filename):
    '''
    Task: Save given model and indexs to disk
    Input: 
        model: Keras model created by _build_network, and trained.
        idx: A dictionary produced by create_indexs, containing word and label indexes, 
             as well as the maximum sentence length. 
        filename: Saves the mode into filename.nn and the indexes into filename.idx
    '''
    model.save("ner-nn.nn")
    
    file = open("index_ner.pkl", "wb")
    pickle.dump(idx, file)
    file.close()

## `Learner()`

In [103]:
def learner(traindir, validationdir):#, modelname):
    '''
    Learns a NN model using traindir as training data, and validationdir as validation data.
    Saves learnt model in a file named modelname
    '''
    # load train and validation data in a suitable form
    train_dataset = load_data(traindir)
    val_dataset = load_data(validationdir)
    
    # create indexes from trainindg data 
    max_len = 100
    idx = create_index(train_dataset, max_len)
    
    # build network 
    model = build_network(idx)
    
    # encode datasets
    Xtrain = encode_words(train_dataset, idx)
    Y, Ytrain = encode_labels(train_dataset, idx)
    Xval = encode_words(val_dataset, idx)
    Yv, Yval = encode_labels(val_dataset, idx)
    
    # train model
    model.fit(Xtrain, np.array(Ytrain),
              batch_size=32,
              epochs=2,
              verbose=1,
              validation_data=(Xval, np.array(Yval)))
    
    # save model and indexs, for later use in prediction
    save_model_and_indexes(model, idx, path_dev.split('/')[-1]+"NER-learned")

In [104]:
path_train = "../../labAHLT/data/train"
path_dev = "../../labAHLT/data/devel"
learner(path_train, path_dev)

Epoch 1/2
170/170 [==============================] - 19s 87ms/step - loss: 0.7273 - accuracy: 0.8701 - val_loss: 0.1534 - val_accuracy: 0.9577
Epoch 2/2
170/170 [==============================] - 14s 81ms/step - loss: 0.1430 - accuracy: 0.9636 - val_loss: 0.1214 - val_accuracy: 0.9704
INFO:tensorflow:Assets written to: ner-nn.nn/assets


***
## Functions Classifier

In [105]:
def load_model_and_indexs():
    '''
    Task: Load model and associate indexs from disk.
    Input:
        filename: filename to be loaded
    Output: Loads a model from filename.nn and its indexes from filename.idx
            Returns the loaded model and indexes. 
    '''
    model = load_model("ner-nn.nn")
    index = open("index_ner.pkl", "rb")
    idx = pickle.load(index)
    
    return model, idx

In [158]:
def output_entities(dataset, preds, outfile):
    '''
    Task: Output detected entities in the format expected by the evaluator
    Input: 
        dataset: A dataset produced by load_data.
        preds: For each sentence in dataset, a list with the labels for each sentence token, 
               as predicted by the model.
    Output: prints the detected entities to stdout in the format required by the evaluator. 
    '''
    outf = open(outfile, 'w')
    for sentence, pred in zip(dataset.items(), preds):
#         print(sentence, pred)
        sid = sentence[0]
        for token, label in zip(sentence[1], pred):
            if label != '<PAD>':
                offset_from = str(token[1])
                offset_to = str(token[2])
                entity = token[0]
                tag_name = label
                outf.write(sid + "|" + offset_from + '-' + offset_to + "|" + entity + "|" + tag_name)
                print(sid + "|" + offset_from + '-' + offset_to + "|" + entity + "|" + tag_name)

In [159]:
#Y = model.predict(X_dev)
#Y = np.argmax(Y, axis=-1)
key_list = list(idx['labels'].keys())
val_list = list(idx['labels'].values())
for y in Y:
    for t in y:
        if t in [2,3,4,5,6,7,8,9]: 
            print(key_list[t])
        break

B-brand
B-brand
B-group
B-group
B-group
B-group
B-group
B-brand
B-brand
B-drug_n
B-drug
B-drug
B-drug
B-drug
B-drug
B-drug
B-drug
B-drug
B-drug
B-drug
I-drug
B-drug
B-drug
B-drug
B-drug
B-group
B-drug
B-drug
B-drug
B-drug
B-group
B-drug
B-drug
B-drug
B-drug
B-drug
B-group
B-drug
B-group
B-group
B-drug
B-drug
B-drug
B-drug
B-drug
B-drug
B-drug
B-drug
B-drug
B-group
B-drug
B-drug
B-drug
B-group
B-brand
B-drug
B-group
B-group
B-group
B-brand
B-drug_n
B-drug
B-drug
I-drug_n
B-drug
B-drug
B-drug
B-drug
B-group
B-drug
B-group
B-group
B-drug
B-group
B-group
B-group
B-drug
B-group
B-drug
B-drug
B-drug
B-drug
B-drug
I-brand
B-brand
B-group
B-group
B-group
B-drug
B-drug
B-drug
B-drug
B-drug
B-drug
B-drug
B-drug
B-drug
B-drug
B-group
I-drug
B-drug
B-group
I-drug
B-group
I-group
B-drug
I-group
I-group
B-brand
I-brand
B-drug_n
B-drug_n
B-brand
B-brand
B-drug
B-drug
B-drug
B-drug
B-drug
B-group
B-drug
B-drug
B-drug
B-drug
B-drug
B-drug
B-group
B-drug
B-drug
B-group
B-drug
B-brand
B-group
B-brand
B-d

## `Classifier()`

In [160]:
def predict(modelname, datadir, outfile):
    '''
    Loads a NN model from a file 'modelname' and uses it to extract drugs in datadir. Saves
    results to 'outfile' in the appropriate format
    '''
    
    # load model and associated encoding data
    model, idx = load_model_and_indexs()
    
    # load data to annotate
    testdata = load_data(datadir)
    
    # encode dataset
    X = encode_words(testdata, idx)
    
    # tag sentences in dataset
    Y = model.predict(X)
    Y = [[find_label(idx, np.argmax(y)) for y in s] for s in Y]
    
    # extract entities and dump them to output file
    output_entities(testdata, Y, outfile)
    
    # evaluate using official evaluator
    evaluation(datadir, outfile)

In [161]:
def find_label(idx, predicted):
    for label, i in idx["labels"].items():
        if i == predicted:
            return label
    

In [162]:
path_test = "../../labAHLT/data/test"
predict(None, path_test, "NER-result")

DDI-DrugBank.d485.s0|0-6|Ethinyl|O
DDI-DrugBank.d485.s0|8-16|estradiol|O
DDI-DrugBank.d485.s0|19-27|Substrate|O
DDI-DrugBank.d485.s0|32-37|CYP3A4|O
DDI-DrugBank.d485.s0|40-44|major|O
DDI-DrugBank.d485.s0|48-52|3A5-7|O
DDI-DrugBank.d485.s0|55-59|minor|O
DDI-DrugBank.d485.s1|0-7|Inhibits|O
DDI-DrugBank.d485.s1|9-14|CYP1A2|O
DDI-DrugBank.d485.s1|17-20|weak|O
DDI-DrugBank.d485.s1|24-26|2B6|O
DDI-DrugBank.d485.s1|17-20|weak|O
DDI-DrugBank.d485.s1|36-39|2C19|O
DDI-DrugBank.d485.s1|17-20|weak|O
DDI-DrugBank.d485.s1|49-51|3A4|O
DDI-DrugBank.d485.s1|17-20|weak|O
DDI-DrugBank.d485.s1|59-59|.|O
DDI-DrugBank.d485.s2|0-12|Acetaminophen|O
DDI-DrugBank.d485.s2|15-17|May|O
DDI-DrugBank.d485.s2|19-26|increase|O
DDI-DrugBank.d485.s2|28-33|plasma|O
DDI-DrugBank.d485.s2|35-47|concentration|O
DDI-DrugBank.d485.s2|52-60|synthetic|O
DDI-DrugBank.d485.s2|62-70|estrogens|O
DDI-DrugBank.d485.s2|73-80|possibly|O
DDI-DrugBank.d485.s2|85-94|inhibiting|O
DDI-DrugBank.d485.s2|96-106|conjugation|O
DDI-DrugBank.d485.s

DDI-DrugBank.d258.s7|74-97|Descarboethoxyloratadine|O
DDI-DrugBank.d258.s7|99-103|After|O
DDI-DrugBank.d258.s7|105-106|10|O
DDI-DrugBank.d258.s7|108-111|Days|O
DDI-DrugBank.d258.s7|116-131|Coadministration|O
DDI-DrugBank.d258.s7|59-68|Loratadine|O
DDI-DrugBank.d258.s7|105-106|10|O
DDI-DrugBank.d258.s7|148-149|mg|O
DDI-DrugBank.d258.s7|155-160|Normal|O
DDI-DrugBank.d258.s7|162-171|Volunteers|O
DDI-DrugBank.d258.s8|0-9|Loratadine|O
DDI-DrugBank.d258.s8|11-34|Descarboethoxyloratadine|O
DDI-DrugBank.d258.s9|0-11|Erythromycin|O
DDI-DrugBank.d258.s9|14-16|500|O
DDI-DrugBank.d258.s9|18-19|mg|O
DDI-DrugBank.d258.s9|21-23|q8h|O
DDI-DrugBank.d258.s9|26-26|+|O
DDI-DrugBank.d258.s9|28-29|40|O
DDI-DrugBank.d258.s9|30-30|%|O
DDI-DrugBank.d258.s9|32-34|+46|O
DDI-DrugBank.d258.s9|30-30|%|O
DDI-DrugBank.d258.s10|0-9|Cimetidine|O
DDI-DrugBank.d258.s10|12-14|300|O
DDI-DrugBank.d258.s10|16-17|mg|O
DDI-DrugBank.d258.s10|19-21|qid|O
DDI-DrugBank.d258.s10|24-27|+103|O
DDI-DrugBank.d258.s10|28-28|%|O
DDI-Drug

DDI-DrugBank.d231.s6|0-9|Furosemide|O
DDI-DrugBank.d231.s6|11-13|may|O
DDI-DrugBank.d231.s6|15-17|add|O
DDI-DrugBank.d231.s6|25-34|potentiate|O
DDI-DrugBank.d231.s6|40-50|therapeutic|O
DDI-DrugBank.d231.s6|52-57|effect|O
DDI-DrugBank.d231.s6|68-83|antihypertensive|O
DDI-DrugBank.d231.s6|85-89|drugs|O
DDI-DrugBank.d231.s6|90-90|.|O
DDI-DrugBank.d231.s7|0-11|Potentiation|O
DDI-DrugBank.d231.s7|13-18|occurs|O
DDI-DrugBank.d231.s7|25-34|ganglionic|O
DDI-DrugBank.d231.s7|39-48|peripheral|O
DDI-DrugBank.d231.s7|50-59|adrenergic|O
DDI-DrugBank.d231.s7|61-68|blocking|O
DDI-DrugBank.d231.s7|70-74|drugs|O
DDI-DrugBank.d231.s7|75-75|.|O
DDI-DrugBank.d231.s8|0-9|Furosemide|O
DDI-DrugBank.d231.s8|11-13|may|O
DDI-DrugBank.d231.s8|15-22|decrease|O
DDI-DrugBank.d231.s8|24-31|arterial|O
DDI-DrugBank.d231.s8|33-46|responsiveness|O
DDI-DrugBank.d231.s8|51-64|norepinephrine|O
DDI-DrugBank.d231.s8|65-65|.|O
DDI-DrugBank.d231.s9|0-6|However|O
DDI-DrugBank.d231.s9|9-22|norepinephrine|O
DDI-DrugBank.d231.s9|2

DDI-DrugBank.d194.s1|180-183|body|O
DDI-DrugBank.d194.s1|185-193|clearance|O
DDI-DrugBank.d194.s1|92-100|etoposide|O
DDI-DrugBank.d194.s1|208-215|compared|O
DDI-DrugBank.d194.s1|92-100|etoposide|O
DDI-DrugBank.d194.s1|230-234|alone|O
DDI-DrugBank.d194.s1|235-235|.|O
DDI-DrugBank.d473.s0|0-7|Although|O
DDI-DrugBank.d473.s0|12-19|clinical|O
DDI-DrugBank.d473.s0|21-29|drug-drug|O
DDI-DrugBank.d473.s0|31-41|interaction|O
DDI-DrugBank.d473.s0|43-49|studies|O
DDI-DrugBank.d473.s0|61-69|conducted|O
DDI-DrugBank.d473.s0|74-77|date|O
DDI-DrugBank.d473.s0|87-91|basis|O
DDI-DrugBank.d473.s0|103-107|vitro|O
DDI-DrugBank.d473.s0|43-49|studies|O
DDI-DrugBank.d473.s0|118-127|cytochrome|O
DDI-DrugBank.d473.s0|129-132|p450|O
DDI-DrugBank.d473.s0|134-143|inhibitors|O
DDI-DrugBank.d473.s0|149-156|inducers|O
DDI-DrugBank.d473.s0|162-169|unlikely|O
DDI-DrugBank.d473.s0|174-179|affect|O
DDI-DrugBank.d473.s0|185-194|metabolism|O
DDI-DrugBank.d473.s0|199-209|clofarabine|O
DDI-DrugBank.d473.s0|210-210|.|O
DDI-

DDI-MedLine.d184.s8|84-90|context|O
DDI-MedLine.d184.s8|92-99|compared|O
DDI-MedLine.d184.s8|106-120|quinpirole-free|O
DDI-MedLine.d184.s8|122-129|controls|O
DDI-MedLine.d184.s8|131-141|conditioned|O
DDI-MedLine.d184.s8|65-75|amphetamine|O
DDI-MedLine.d184.s8|170-175|female|O
DDI-MedLine.d184.s8|122-129|controls|O
DDI-MedLine.d184.s8|131-141|conditioned|O
DDI-MedLine.d184.s8|65-75|amphetamine|O
DDI-MedLine.d184.s8|52-56|spent|O
DDI-MedLine.d184.s8|47-50|time|O
DDI-MedLine.d184.s8|238-248|drug-paired|O
DDI-MedLine.d184.s8|84-90|context|O
DDI-MedLine.d184.s8|92-99|compared|O
DDI-MedLine.d184.s8|272-285|saline-treated|O
DDI-MedLine.d184.s8|122-129|controls|O
DDI-MedLine.d184.s8|295-295|.|O
DDI-MedLine.d184.s9|0-8|Increased|O
DDI-MedLine.d184.s9|10-10|D|O
DDI-MedLine.d184.s9|14-18|-like|O
DDI-MedLine.d184.s9|20-27|receptor|O
DDI-MedLine.d184.s9|29-39|sensitivity|O
DDI-MedLine.d184.s9|41-47|appears|O
DDI-MedLine.d184.s9|57-64|enhanced|O
DDI-MedLine.d184.s9|70-79|behavioral|O
DDI-MedLine.d18

DDI-DrugBank.d585.s0|104-117|antiarrhythmic|O
DDI-DrugBank.d585.s0|14-19|agents|O
DDI-DrugBank.d585.s0|129-133|class|O
DDI-DrugBank.d585.s0|135-135|I|O
DDI-DrugBank.d585.s0|138-141|e.g.|O
DDI-DrugBank.d585.s0|144-152|quinidine|O
DDI-DrugBank.d585.s0|156-169|antihistamines|O
DDI-DrugBank.d585.s0|172-184|antipsychotic|O
DDI-DrugBank.d585.s0|14-19|agents|O
DDI-DrugBank.d585.s0|138-141|e.g.|O
DDI-DrugBank.d585.s0|200-213|phenothiazines|O
DDI-DrugBank.d585.s0|217-231|benzodiazepines|O
DDI-DrugBank.d585.s0|234-236|MAO|O
DDI-DrugBank.d585.s0|238-247|inhibitors|O
DDI-DrugBank.d585.s0|250-257|narcotic|O
DDI-DrugBank.d585.s0|259-268|analgesics|O
DDI-DrugBank.d585.s0|138-141|e.g.|O
DDI-DrugBank.d585.s0|277-286|meperidine|O
DDI-DrugBank.d585.s0|290-297|nitrates|O
DDI-DrugBank.d585.s0|303-310|nitrites|O
DDI-DrugBank.d585.s0|313-327|sympathomimetic|O
DDI-DrugBank.d585.s0|14-19|agents|O
DDI-DrugBank.d585.s0|337-345|tricyclic|O
DDI-DrugBank.d585.s0|347-361|antidepressants|O
DDI-DrugBank.d585.s0|85-89|

NameError: name 'evaluation' is not defined